# Técnicas para el almacenamiento y consulta de datos estructurados

Structured Query Language o SQL es un lenguaje de programación utilizado en la creación, administración y manipulación de bases de datos relacionales. Este es un lenguaje de alto nivel, por lo que su sintasxis es fácilmente entendible.

Una base de base de datos relacional es una manera de almacenar y manipular datos. En este tipo de bases de datos, los datos se encuentran almacenados en forma de tablas. Cada columna tiene columnas, también conocidas como campos, que describen a los datos; y filas, también conocidas como registros, los cuales contienen datos.

Cuando deseamos hacer una consulta de datos en una base de datos debemos enviar una serie de comandos al administrador de la base de datos (*database manager*), estos se conocen como *queries*. El administrador de base de datos recibe la consulta y realiza los cálculos indicados en la *query*, una vez completados devuelve los resultados en una forma tabular. Existen diferentes administradores de bases de datos, en esta sesión utilizaremos *sqlite*.

A lo largo de este libreta exploraremos la información contenida en una base de datos empleando *queries* SQL a través de Python.

## Objetivos de Aprendizaje

Al final de la lección, el participantes será capaz de:
* Utilizar queries para extraer datos en bases de datos relacionales.
* Realizar agrupamientos y agregaciones a partir de los datos para la creación de métricas.
* Combinar datos de diferentes tablas.

## Contenido

1. Acceso a los Datos con Queries.
2. Agregaciones y Agrupamientos de Datos.
3. Combinación de Tablas.

## 1. Acceso a los Datos con Queries

A fines de la década de 1920 y principios de la de 1930, William Dyer, Frank Pabodie y Valentina Roerich dirigieron expediciones al Polo de Inaccesibilidad en el Pacífico Sur y luego a la Antártida. Hace unos años, sus expediciones fueron encontradas en un casillero de en la Universidad de Miskatonic[1](https://swcarpentry.github.io/sql-novice-survey/). Los datos de estas exploraciones se han guardado en `survey.db`.

La base de datos está integrado por las siguientes tablas:
* Person: Personas que tomaron lecturas, siendo `id` el identificador único de esa persona.

|id	|personal	|family
|--|--|--
|dyer	|William	|Dyer
|pb	|Frank	|Pabodie
|lake	|Anderson	|Lake
|roe	|Valentina	|Roerich
|danforth	|Frank	|Danforth

* Survey: Las medidas tomadas en cada ubicación precisa en estos sitios. Se identifican por el índice `taken`. El campo`quant` es la abreviatura de cantidad (*quantity*) e indica lo que se está midiendo. Los valores son `rad`, `sal` y `temp` que se refieren a 'radiación', 'salinidad' y 'temperatura', respectivamente.


|taken	|person	|quant	|reading
|--|--|--|--
|619	|dyer	|rad	|9.82
|619	|dyer	|sal	|0.13
|622	|dyer	|rad	|7.8
|622	|dyer	|sal	|0.09


* Site: Ubicaciones de los sitios donde se tomaron las lecturas.

|name	|lat	|long
|--|--|--
|DR-1	|-49.85	|-128.57
|DR-3	|-47.15	|-126.72
|MSK-4	|-48.87	|-123.4

* Visited: Identificación específica `id` de las ubicaciones precisas donde se tomaron las lecturas en los sitios y fechas.

|id	|site	|dated
|--|--|--
|619	|DR-1	|1927-02-08
|622	|DR-1	|1927-02-10
|734	|DR-3	|1930-01-07
|735	|DR-3	|1930-01-12
|751	|DR-3	|1930-02-26

### 1.1 Conexión a la Base de Datos

Python provee una biblioteca estándar que permite conectarse al administrador de base de datos *sqlite*. De modo que simplemente es necesario importarla en nuestra libreta de trabajo.

In [68]:
import sqlite3

Una vez que hemos importado el módulo en nuestra libreta de trabajo, es necesario identificar la dirección en la cual se encuentra la base de dato para poder establecer la conexión.

La conexión puede completarse con la propiedad `connect` de `sqlite3`. El comando `sqlite3.connect()` es el que se utiliza para establecer una conección. Simplemente se requiere pasar como parámetros la base de datos que se desea emplear.

In [69]:
connection = sqlite3.connect("survey.db")

In [70]:
type(connection)

sqlite3.Connection

Si `sqlite3` no encuentra la base de datos a la que se está intentando acceder, entonces creará una bajo el nombre indicado; y la guardará en el directorio de trabajo.

La interacción entre Python y SQLite3 se completará a través de los cursores. Estos son objetos de Python cuyos métodos permiten ejecutar comandos propios de SQL tales como búsquedas, actualizaciones, filtros, entre otros. Desde una perspectiva técnica, estos son punteros a una base de datos que realiza un seguimiento de las operaciones pendientes.

In [71]:
cursor = connection.cursor()

El método `cursos.execute()` puede emplearse para que el cursos solicite a la base de datos ejecutar una *query*. La *queyr*, escrita en SQL, debe pasarse al método anterior en formato de cadena de caracteres. En la siguiente celda se ejecuta la *query* `SELECT * FROM Person`, lo cual permitirá obtener todas las columnas de la tabla `Person`.

La base de datos nos devuelve los resultados de la consulta en respuesta a la llamada `cursor.fetchall()`. Este resultado es una lista con una entrada para cada registro en el conjunto de resultados, si recorremos esa lista e imprimimos esas entradas de la lista, podemos ver que cada una es una tupla con un elemento para cada campo solicitado.

In [72]:
cursor.execute("SELECT * FROM Person")
results = cursor.fetchall()
for r in results:
    print(r)

('dyer', 'William', 'Dyer')
('pb', 'Frank', 'Pabodie')
('lake', 'Anderson', 'Lake')
('roe', 'Valentina', 'Roerich')
('danforth', 'Frank', 'Danforth')


Una vez que hemos terminado de trabajar con alguna base de datos específica, será necesario cerrar las conexiones establecidas.

In [73]:
cursor.close()
connection.close()

### 3.2 Selección de Datos

La query más básica en SQL permite la selección de una columna de una tabla específica, para ello será necesario:
* indicar la columna que se desea obtener después de la palabra clave `SELECT`
* indicar la tabla después de la palabra clave `FROM`

De modo que si se desea seleccionar la columna `personal` de la tabla `Person` se emplearía la siguiente query:

In [74]:
connection = sqlite3.connect("survey.db")
cursor = connection.cursor()

In [75]:
query = """
        SELECT personal
        FROM person
        """

In [76]:
cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('William',)
('Frank',)
('Anderson',)
('Valentina',)
('Frank',)


Las palabras clave `SELECT` y `FROM` se han colocado en mayúsculas. Sin embargo, SQL es insensible ante las mayúsculas y minúsculas, pero es útil desde el punto de vista de la lectura y estilo de programación.

Si deseamos obtener más información, podemos añadir más columnas a la lista de campos después del `SELECT`.

In [77]:
query = """
        SELECT personal, family
        FROM Person
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('William', 'Dyer')
('Frank', 'Pabodie')
('Anderson', 'Lake')
('Valentina', 'Roerich')
('Frank', 'Danforth')


Si desea mostrar todas las columnas de una tabla, utilice el `*` después del `SELECT`.

In [78]:
query = """
        SELECT *
        FROM Person
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('dyer', 'William', 'Dyer')
('pb', 'Frank', 'Pabodie')
('lake', 'Anderson', 'Lake')
('roe', 'Valentina', 'Roerich')
('danforth', 'Frank', 'Danforth')


La siguiente query permite acceder a las columnas `taken`, `quant` y `reading` de la tabla `Survey`.

In [79]:
query = """
        SELECT taken, quant, reading
        FROM Survey
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(619, 'rad', 9.82)
(619, 'sal', 0.13)
(622, 'rad', 7.8)
(622, 'sal', 0.09)
(734, 'rad', 8.41)
(734, 'temp', -21.5)
(735, 'rad', 7.22)
(735, 'sal', 0.06)
(735, 'temp', -26.0)
(751, 'rad', 4.35)
(751, 'sal', 0.1)
(751, 'temp', -18.5)
(752, 'rad', 4.2)
(752, 'sal', 0.09)
(752, 'temp', -16.0)
(756, 'rad', 7.51)
(756, 'sal', 0.05)
(756, 'temp', -17.0)
(757, 'rad', 6.27)
(757, 'sal', 0.04)
(757, 'temp', -26.5)
(758, 'rad', 5.83)
(758, 'sal', 0.06)
(758, 'temp', -29.0)
(761, 'rad', 8.34)
(761, 'sal', 0.05)
(761, 'temp', -24.0)
(762, 'rad', 7.11)
(762, 'sal', 0.05)
(762, 'temp', -25.5)
(763, 'rad', 8.64)
(763, 'sal', 0.05)
(763, 'temp', -21.5)


En algunas ocasiones, no deseamos ver todos los resultados. En este caso puede emplearse el comando `LIMIT`para limitar la cantidad de registros mostrados. Esto puede resultar particularmente útil cuando se está trabajando con conjunto de datos grandes.

In [80]:
query = """
        SELECT taken, quant, reading
        FROM Survey
        LIMIT 10
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(619, 'rad', 9.82)
(619, 'sal', 0.13)
(622, 'rad', 7.8)
(622, 'sal', 0.09)
(734, 'rad', 8.41)
(734, 'temp', -21.5)
(735, 'rad', 7.22)
(735, 'sal', 0.06)
(735, 'temp', -26.0)
(751, 'rad', 4.35)


Recordemos que la columna `quant` contiene la cantidad que fue medida. Anteriormente, se indicó que los tres elementos medidos son la radiación, la salinidad y la temperatura. Supongamos que no supieramos y quisieramos desplegar los valores únicos en esa columna. En tal caso podríamos utilizar `DISTINCT` después del comando `SELECT`. Efectivamente, observamos que los valores anteriormente indicados son los valores únicos en esa columna.

In [81]:
query = """
        SELECT DISTINCT quant
        FROM Survey
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('rad',)
('sal',)
('temp',)


De igual manera, podemos añadir condiciones de filtrado a nuestra query, es decir, seleccionar aquellos registros que cumplen un criterio específico. En este caso se emplearía `WHERE`. El criterio a cumplir se coloca después de esta palabra clave.

Por ejemplo, consideremos que queremos observar únicamente las mediciones de temperatura. En tal caso, procederíamos como sigue:

In [82]:
query = """
        SELECT *
        FROM Survey
        WHERE quant = 'temp'
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(734, 'lake', 'temp', -21.5)
(735, 'lake', 'temp', -26.0)
(751, 'roe', 'temp', -18.5)
(752, 'roe', 'temp', -16.0)
(756, 'danforth', 'temp', -17.0)
(757, 'danforth', 'temp', -26.5)
(758, 'scobie', 'temp', -29.0)
(761, 'lake', 'temp', -24.0)
(762, 'lake', 'temp', -25.5)
(763, 'roe', 'temp', -21.5)


Podemos crear criterios de selección más sofisticados al combinar sentencias lógicas a través de los operadores lógicos `AND` y `OR`.

Por ejemplo, consideremos que queremos las medidas de temperatura que son menos a -20.0 C.

In [83]:
query = """
        SELECT *
        FROM Survey
        WHERE (quant = 'temp') AND (reading < -20)
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(734, 'lake', 'temp', -21.5)
(735, 'lake', 'temp', -26.0)
(757, 'danforth', 'temp', -26.5)
(758, 'scobie', 'temp', -29.0)
(761, 'lake', 'temp', -24.0)
(762, 'lake', 'temp', -25.5)
(763, 'roe', 'temp', -21.5)


Los paréntesis en la query anterior no son necesarios. Sin embargo, aportan facilidad de lectura en el código.

Supongamos que deseamos obtener las medidas hechas por William Dyer (`dyer`) y Frank Pabodie (`pb`). Esto puede obtenerse utilizando el operador `OR`.

In [84]:
query = """
        SELECT *
        FROM Survey
        WHERE (person = 'dyer') OR (person = 'pb')
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(619, 'dyer', 'rad', 9.82)
(619, 'dyer', 'sal', 0.13)
(622, 'dyer', 'rad', 7.8)
(622, 'dyer', 'sal', 0.09)


Sin embargo, en el caso que tuvieramos muchos investigadores, esto sería impráctico. En este caso podemos utilizar `IN` para indicar la selección de aquellos valores que estén contenidos en una lista dada.

De esta manera, la query anterior podría completarse como sigue.

In [85]:
query = """
        SELECT *
        FROM Survey
        WHERE person IN ('dyer','pb')
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(619, 'dyer', 'rad', 9.82)
(619, 'dyer', 'sal', 0.13)
(622, 'dyer', 'rad', 7.8)
(622, 'dyer', 'sal', 0.09)


Observemos los registros en la tabla `Visited`. Vemos como se tienen 8 registros, de los cuales uno tiene un valor nulo.

In [86]:
query = """
        SELECT *
        FROM Visited
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(619, 'DR-1', '1927-02-08')
(622, 'DR-1', '1927-02-10')
(734, 'DR-3', '1930-01-07')
(735, 'DR-3', '1930-01-12')
(751, 'DR-3', '1930-02-26')
(752, 'MSK-4', None)
(756, 'DR-1', '1930-03-22')
(757, 'DR-1', '1930-03-25')
(758, 'MSK-4', '1930-03-29')
(761, 'DR-1', '1930-04-01')
(762, 'DR-3', '1930-04-22')
(763, 'DR-3', '1930-04-25')


En ocasiones, las columnas de fecha ya tienen un forma apropiado dentro de las bases de datos y la información no está únicamente guardada en forma de cadenas de caracteres. Debido a esto, es posible aplicar directamente filtros respectando la estructura de las mismas. A continuación se seleccionan las visitas que son posteriores al primero de enero de 1930.

In [87]:
query = """
        SELECT *
        FROM Visited
        WHERE dated > '1930-01-01'
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(734, 'DR-3', '1930-01-07')
(735, 'DR-3', '1930-01-12')
(751, 'DR-3', '1930-02-26')
(756, 'DR-1', '1930-03-22')
(757, 'DR-1', '1930-03-25')
(758, 'MSK-4', '1930-03-29')
(761, 'DR-1', '1930-04-01')
(762, 'DR-3', '1930-04-22')
(763, 'DR-3', '1930-04-25')


Supongamos que queremos seleccionar los valores válidos, es decir, aquellos cuyo valor sea diferentes a nulo. Esto puede completarse considerando que los valores nulos se identifican con `NULL`. Sin embargo, los filtros empleando `date = NULL` o `date != NULL` no funcionarán. En este caso se debe utilizar la prueba especial `IS NULL` o `IS NOT NULL`.

In [88]:
query = """
        SELECT *
        FROM Visited
        WHERE dated IS NOT null
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(619, 'DR-1', '1927-02-08')
(622, 'DR-1', '1927-02-10')
(734, 'DR-3', '1930-01-07')
(735, 'DR-3', '1930-01-12')
(751, 'DR-3', '1930-02-26')
(756, 'DR-1', '1930-03-22')
(757, 'DR-1', '1930-03-25')
(758, 'MSK-4', '1930-03-29')
(761, 'DR-1', '1930-04-01')
(762, 'DR-3', '1930-04-22')
(763, 'DR-3', '1930-04-25')


In [89]:
query = """
        SELECT *
        FROM Visited
        WHERE dated IS null
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(752, 'MSK-4', None)


### 3.3 Ordenamiento

Los resultados de las queries pueden ordenarse utilizando `ORDER BY`. Por ejemplo, anteriormente se había obtenido únicamente las mediciones relacionadas con la temperature de la tabla `Survey`. Esta misma query puede presentarse, pero ahora con los valores de temperatura ordenados de forma creciente.

In [90]:
query = """
        SELECT *
        FROM Survey
        WHERE quant = 'temp'
        ORDER BY reading
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(758, 'scobie', 'temp', -29.0)
(757, 'danforth', 'temp', -26.5)
(735, 'lake', 'temp', -26.0)
(762, 'lake', 'temp', -25.5)
(761, 'lake', 'temp', -24.0)
(734, 'lake', 'temp', -21.5)
(763, 'roe', 'temp', -21.5)
(751, 'roe', 'temp', -18.5)
(756, 'danforth', 'temp', -17.0)
(752, 'roe', 'temp', -16.0)


Por defecto, el ordenamiento se hace de forma ascendente. El orden puede indicarse a través de los argumentos `ASC` y `DESC` para ascendente y descendente, respectivamente.

In [91]:
query = """
        SELECT *
        FROM Survey
        WHERE quant = 'temp'
        ORDER BY reading ASC
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(758, 'scobie', 'temp', -29.0)
(757, 'danforth', 'temp', -26.5)
(735, 'lake', 'temp', -26.0)
(762, 'lake', 'temp', -25.5)
(761, 'lake', 'temp', -24.0)
(734, 'lake', 'temp', -21.5)
(763, 'roe', 'temp', -21.5)
(751, 'roe', 'temp', -18.5)
(756, 'danforth', 'temp', -17.0)
(752, 'roe', 'temp', -16.0)


In [92]:
query = """
        SELECT *
        FROM Survey
        WHERE quant = 'temp'
        ORDER BY reading DESC
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(752, 'roe', 'temp', -16.0)
(756, 'danforth', 'temp', -17.0)
(751, 'roe', 'temp', -18.5)
(734, 'lake', 'temp', -21.5)
(763, 'roe', 'temp', -21.5)
(761, 'lake', 'temp', -24.0)
(762, 'lake', 'temp', -25.5)
(735, 'lake', 'temp', -26.0)
(757, 'danforth', 'temp', -26.5)
(758, 'scobie', 'temp', -29.0)


De igual manera podemos order considerando varios campos. Por ejemplo, supongamos que queremos ordenar los nombres de los científicos de forma alfabética contemplando primero el nombre (`personal`) y luego el apellido (`family`). En tal caso, la query sería como sigue.

In [93]:
query = """
        SELECT personal, family
        FROM Person
        ORDER BY personal, family
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('Anderson', 'Lake')
('Frank', 'Danforth')
('Frank', 'Pabodie')
('Valentina', 'Roerich')
('William', 'Dyer')


Las columnas empleadas en el ordenamiento, no tienen que seguir el mismo criterio de crecimiento.

In [94]:
query = """
        SELECT personal, family
        FROM Person
        ORDER BY personal ASC, family DESC
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('Anderson', 'Lake')
('Frank', 'Pabodie')
('Frank', 'Danforth')
('Valentina', 'Roerich')
('William', 'Dyer')


De hecho, ¡no se requiere devolver una columna que ha sido utilizada para ordenar una tabla! En la siguiente query, los datos son ordenados bajo el criterio anteriormente descrito, pero únicamente se devuelve el `id`.

In [95]:
query = """
        SELECT id
        FROM Person
        ORDER BY personal ASC, family DESC
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('lake',)
('pb',)
('danforth',)
('roe',)
('dyer',)


El ejercicio anterior nos hace reflexionar lo siguiente. El orden en el cual las clausulas son escritas usualmente es `SELECT`, `FROM`, `WHERE`y `ORDER BY`. Sin embargo, este no es el orden en que se ejecutan. En este caso en particular primero se realiza el filtrado de las columnas de acuerdo al `WHERE`, posteriormente se realiza el ordenamiento `ORDER BY` y finalmente se despliegan las columnas o expresiones indicadas en `SELECT`.

## 2. Agregaciones y Agrupamiento de Datos

### 2.1 Cálculo de Nuevos Valores

La temperatura en la tabla `Survey` se encuentra en Farenheit. Consideremos que deseamos crear una nueva columna con los valores convertidos a Celsius. Esto puede completarse como sigue.

In [96]:
query = """
        SELECT taken, 5 * (reading - 32) / 9
        FROM Survey
        WHERE quant = 'temp'
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(734, -29.72222222222222)
(735, -32.22222222222222)
(751, -28.055555555555557)
(752, -26.666666666666668)
(756, -27.22222222222222)
(757, -32.5)
(758, -33.888888888888886)
(761, -31.11111111111111)
(762, -31.944444444444443)
(763, -29.72222222222222)


Sin embargo, observamos que se tienen muchos decimales en los valores de la temperatura en Celsius. Utilicemos `ROUND` para indicar únicamente 2 posiciones decimales.

In [97]:
query = """
        SELECT taken, ROUND(5 * (reading - 32) / 9, 2)
        FROM Survey
        WHERE quant = 'temp'
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(734, -29.72)
(735, -32.22)
(751, -28.06)
(752, -26.67)
(756, -27.22)
(757, -32.5)
(758, -33.89)
(761, -31.11)
(762, -31.94)
(763, -29.72)


SQL nos permite renombrar los campos indicando un pseudónimo mediante `AS`. De modo que es posible indicar explícitamente que la temperatura se encuentra en celsius.

In [98]:
query = """
        SELECT taken, ROUND(5 * (reading - 32) / 9, 2) AS Celsius
        FROM Survey
        WHERE quant = 'temp'
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(734, -29.72)
(735, -32.22)
(751, -28.06)
(752, -26.67)
(756, -27.22)
(757, -32.5)
(758, -33.89)
(761, -31.11)
(762, -31.94)
(763, -29.72)


Por otro lado, supongamos que queremos unir `personal` con `family` para obtener el nombre completo de los científicos. Esto puede completarse utilizando el operador de concatenación `||`.

In [99]:
query = """
        SELECT personal || ' ' || family
        FROM Person;
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('William Dyer',)
('Frank Pabodie',)
('Anderson Lake',)
('Valentina Roerich',)
('Frank Danforth',)


### 2.2 Agregaciones

Las agregaciones nos permiten el cálculo de valores combinados en grupos.

Por ejemplo, se ha utilizado `COUNT` en la siguiente query para realizar el cálculo de la cantidad de registros en las mediciones de la tabla `Survey`.

In [100]:
query = """
        SELECT COUNT(*)
        FROM Survey
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(33,)


¿Cuántos registros corresponden a la temperatura y cuál es su valor promedio? Esto puede resolverse como sigue.

In [101]:
query = """
        SELECT COUNT(reading), AVG(reading)
        FROM Survey
        WHERE quant = 'temp'
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(10, -22.55)


Existen otros métodos de agregación en SQL como `MAX`, `MIN` y `AVG`. Los valores mínimos y máximos de las fechas en que se hicieron las visitas se obtienen de la siguiente manera.

In [102]:
query = """
        SELECT MIN(dated), MAX(dated)
        FROM Visited
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('1927-02-08', '1930-04-25')


### 3.3 Agrupaciones

Considera que se desea obtener el promedo de las mediciones de temperatura hecha por cada científico. En este caso será necesario utilizar `GROUP BY`. Este funciona de manera análoga al discutido anteriormente en Pandas: agrupa los registros que poseen valores igualesde una columna específica. Una vez que los elementos han sido agrupados, es posible agregar las columnas restantes.

In [103]:
query = """
        SELECT person, reading
        FROM Survey
        WHERE quant = 'temp'
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('lake', -21.5)
('lake', -26.0)
('roe', -18.5)
('roe', -16.0)
('danforth', -17.0)
('danforth', -26.5)
('scobie', -29.0)
('lake', -24.0)
('lake', -25.5)
('roe', -21.5)


A continuación se tiene una query que obtiene el nombre del científico, la medición promedio de la temperatura y el número de mediciones.

In [104]:
query = """
        SELECT person, ROUND(AVG(reading), 2), COUNT(reading)
        FROM Survey
        WHERE (quant = 'temp') AND (person IS NOT NULL)
        GROUP BY person
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('danforth', -21.75, 2)
('lake', -24.25, 4)
('roe', -18.67, 3)
('scobie', -29.0, 1)


Supongamos se desea obtener el promedio de la medición hecha de cada científico para cantidad física. En este caso, el `GROUP BY` requerirá formar grupos considerando dos columnas `person` y `quant`.

In [105]:
query = """
        SELECT   person, quant, COUNT(reading), ROUND(AVG(reading), 2)
        FROM     Survey
        WHERE    person IS NOT NULL
        GROUP BY person, quant
        ORDER BY person, quant
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('danforth', 'rad', 2, 6.89)
('danforth', 'sal', 2, 0.05)
('danforth', 'temp', 2, -21.75)
('dyer', 'rad', 2, 8.81)
('dyer', 'sal', 2, 0.11)
('lake', 'rad', 4, 7.77)
('lake', 'sal', 3, 0.05)
('lake', 'temp', 4, -24.25)
('roe', 'rad', 3, 5.73)
('roe', 'sal', 3, 0.08)
('roe', 'temp', 3, -18.67)
('scobie', 'rad', 1, 5.83)
('scobie', 'sal', 1, 0.06)
('scobie', 'temp', 1, -29.0)


## 3. Combinación de Tablas

Considera que se desea obtener una tabla que contenga la latitud, longitud, la fecha, la cantidad medida y su valor. Esto requiere de la integración de resultados de diferentes columnas, ya que las latitudes y longitudes se encuentran en la tabla `Site`, las fechas y las medidas en `Visited` y las lecturas en `Survey`. La siguiente figura muestras las relaciones que se tienen en las diferentes tablas.

![Relaciones de las Tablas](https://swcarpentry.github.io/sql-novice-survey/fig/sql-join-structure.svg)

El comando `JOIN` se emplea junto con `ON`para combinar campos de diferentes tablas. El comando JOIN por sí solo dará como resultado un producto cruzado, donde cada fila de la primera tabla se empareja con cada fila de la segunda tabla.

|id	|personal	|family
|--|--|--
|dyer	|William	|Dyer
|pb	|Frank	|Pabodie
|lake	|Anderson	|Lake
|roe	|Valentina	|Roerich
|danforth	|Frank	|Danforth

* Survey: Las medidas tomadas en cada ubicación precisa en estos sitios. Se identifican por el índice `taken`. El campo`quant` es la abreviatura de cantidad (*quantity*) e indica lo que se está midiendo. Los valores son `rad`, `sal` y `temp` que se refieren a 'radiación', 'salinidad' y 'temperatura', respectivamente.


|taken	|person	|quant	|reading
|--|--|--|--
|619	|dyer	|rad	|9.82
|619	|dyer	|sal	|0.13
|622	|dyer	|rad	|7.8
|622	|dyer	|sal	|0.09


* Site: Ubicaciones de los sitios donde se tomaron las lecturas.

|name	|lat	|long
|--|--|--
|DR-1	|-49.85	|-128.57
|DR-3	|-47.15	|-126.72
|MSK-4	|-48.87	|-123.4

* Visited: Identificación específica `id` de las ubicaciones precisas donde se tomaron las lecturas en los sitios y fechas.

|id	|site	|dated
|--|--|--
|619	|DR-1	|1927-02-08
|622	|DR-1	|1927-02-10
|734	|DR-3	|1930-01-07
|735	|DR-3	|1930-01-12
|751	|DR-3	|1930-02-26

In [106]:
query = """
        SELECT * FROM Site JOIN Visited;
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('DR-1', -49.85, -128.57, 619, 'DR-1', '1927-02-08')
('DR-1', -49.85, -128.57, 622, 'DR-1', '1927-02-10')
('DR-1', -49.85, -128.57, 734, 'DR-3', '1930-01-07')
('DR-1', -49.85, -128.57, 735, 'DR-3', '1930-01-12')
('DR-1', -49.85, -128.57, 751, 'DR-3', '1930-02-26')
('DR-1', -49.85, -128.57, 752, 'MSK-4', None)
('DR-1', -49.85, -128.57, 756, 'DR-1', '1930-03-22')
('DR-1', -49.85, -128.57, 757, 'DR-1', '1930-03-25')
('DR-1', -49.85, -128.57, 758, 'MSK-4', '1930-03-29')
('DR-1', -49.85, -128.57, 761, 'DR-1', '1930-04-01')
('DR-1', -49.85, -128.57, 762, 'DR-3', '1930-04-22')
('DR-1', -49.85, -128.57, 763, 'DR-3', '1930-04-25')
('DR-3', -47.15, -126.72, 619, 'DR-1', '1927-02-08')
('DR-3', -47.15, -126.72, 622, 'DR-1', '1927-02-10')
('DR-3', -47.15, -126.72, 734, 'DR-3', '1930-01-07')
('DR-3', -47.15, -126.72, 735, 'DR-3', '1930-01-12')
('DR-3', -47.15, -126.72, 751, 'DR-3', '1930-02-26')
('DR-3', -47.15, -126.72, 752, 'MSK-4', None)
('DR-3', -47.15, -126.72, 756, 'DR-1', '1930-03-22')
('DR-3

 Por lo general, esto no es lo que se desea cuando se combinan dos tablas con datos que están relacionados de alguna manera. Para eso, necesitamos indicar a la computadora qué columnas proporcionan el enlace entre las dos tablas usando la palabra ON. En la query, `ON` determina qué columna de cada tabla usar para combinar las tablas. La unión de cada registro ed la primera con los de la segunda dan todos las posibles combinaciones. En la tabla `Site` se tienen 3 registros, mientras que en `Visited` se tienen 8, por lo que en la unión se muestran 24 registros. Además, como cada tabla tienen 3 campos, en total se tiene 6 campos. El resultado es una tabla con 24 registros y 6 campos.

Para realizar una correcta unión, necesitamos indicar a la computadora qué columnas proporcionan el enlace entre las dos tablas usando la palabra `ON`.

In [107]:
query = """
        SELECT *
        FROM Site
        JOIN Visited
            ON Site.name = Visited.site
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

('DR-1', -49.85, -128.57, 619, 'DR-1', '1927-02-08')
('DR-1', -49.85, -128.57, 622, 'DR-1', '1927-02-10')
('DR-1', -49.85, -128.57, 756, 'DR-1', '1930-03-22')
('DR-1', -49.85, -128.57, 757, 'DR-1', '1930-03-25')
('DR-1', -49.85, -128.57, 761, 'DR-1', '1930-04-01')
('DR-3', -47.15, -126.72, 734, 'DR-3', '1930-01-07')
('DR-3', -47.15, -126.72, 735, 'DR-3', '1930-01-12')
('DR-3', -47.15, -126.72, 751, 'DR-3', '1930-02-26')
('DR-3', -47.15, -126.72, 762, 'DR-3', '1930-04-22')
('DR-3', -47.15, -126.72, 763, 'DR-3', '1930-04-25')
('MSK-4', -48.87, -123.4, 752, 'MSK-4', None)
('MSK-4', -48.87, -123.4, 758, 'MSK-4', '1930-03-29')


In [108]:
query = """
        SELECT
            Site.lat,
            Site.long,
            Visited.dated
        FROM Site
        JOIN Visited
            ON Site.name = Visited.site
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(-49.85, -128.57, '1927-02-08')
(-49.85, -128.57, '1927-02-10')
(-49.85, -128.57, '1930-03-22')
(-49.85, -128.57, '1930-03-25')
(-49.85, -128.57, '1930-04-01')
(-47.15, -126.72, '1930-01-07')
(-47.15, -126.72, '1930-01-12')
(-47.15, -126.72, '1930-02-26')
(-47.15, -126.72, '1930-04-22')
(-47.15, -126.72, '1930-04-25')
(-48.87, -123.4, None)
(-48.87, -123.4, '1930-03-29')


En este caso, se ha especificado cada columna como `Table.field` debido a que las tablas pueden tener campos con los mismos nombres. El resultado deseado se puede obtener mediante la unión de más tablas.

In [109]:
query = """
        SELECT
          Site.lat,
          Site.long,
          Visited.dated,
          Survey.quant,
          Survey.reading
        FROM
          Site
          JOIN Visited
          JOIN Survey ON Site.name = Visited.site
          AND Visited.id = Survey.taken
          AND Visited.dated IS NOT NULL
        """

cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(-49.85, -128.57, '1927-02-08', 'rad', 9.82)
(-49.85, -128.57, '1927-02-08', 'sal', 0.13)
(-49.85, -128.57, '1927-02-10', 'rad', 7.8)
(-49.85, -128.57, '1927-02-10', 'sal', 0.09)
(-47.15, -126.72, '1930-01-07', 'rad', 8.41)
(-47.15, -126.72, '1930-01-07', 'temp', -21.5)
(-47.15, -126.72, '1930-01-12', 'rad', 7.22)
(-47.15, -126.72, '1930-01-12', 'sal', 0.06)
(-47.15, -126.72, '1930-01-12', 'temp', -26.0)
(-47.15, -126.72, '1930-02-26', 'rad', 4.35)
(-47.15, -126.72, '1930-02-26', 'sal', 0.1)
(-47.15, -126.72, '1930-02-26', 'temp', -18.5)
(-49.85, -128.57, '1930-03-22', 'rad', 7.51)
(-49.85, -128.57, '1930-03-22', 'sal', 0.05)
(-49.85, -128.57, '1930-03-22', 'temp', -17.0)
(-49.85, -128.57, '1930-03-25', 'rad', 6.27)
(-49.85, -128.57, '1930-03-25', 'sal', 0.04)
(-49.85, -128.57, '1930-03-25', 'temp', -26.5)
(-48.87, -123.4, '1930-03-29', 'rad', 5.83)
(-48.87, -123.4, '1930-03-29', 'sal', 0.06)
(-48.87, -123.4, '1930-03-29', 'temp', -29.0)
(-49.85, -128.57, '1930-04-01', 'rad', 8.34)
(-4

Existen diferentes tipos de uniones, a continuación se muestra una imagen con las más relevantes.

![Tipos de Uniones](https://i.pinimg.com/originals/17/a5/2d/17a52d72161197cc9e2a52aa0b875697.png)

## Referencias

* Intro to SQL by Kaggle: https://www.kaggle.com/learn/intro-to-sql.
* Databases and SQL by Software Carpentry: https://swcarpentry.github.io/sql-novice-survey/10-prog/index.html
* Data Management with SQL by Data Carpentry: https://datacarpentry.org/sql-ecology-lesson/03-sql-joins/index.html